In [45]:
# Modules de base
import numpy as np
import os
import sys
import pandas as pd

# Module relatif à Gurobi
from gurobipy import *

module_path = os.path.abspath('..')
sys.path.append(module_path)

from module.utils import *

In [46]:
# Charger le fichier Excel
file = "instance_WPY_realiste_jalon1.xlsx"

# Définition de la base temporelle (08/08/2022 à minuit)
base_time = pd.Timestamp("2022-08-08 00:00")

# Lire les feuilles "Sillons arrivée" et "Sillons départ"
df_sillons_arr = pd.read_excel(file, sheet_name="Sillons arrivee")  # Arrivées
df_sillons_dep = pd.read_excel(file, sheet_name="Sillons depart")  # Départs

# Conversion des dates en datetime64
df_sillons_arr["JARR"] = pd.to_datetime(df_sillons_arr["JARR"], format="%d/%m/%Y", errors="coerce")
df_sillons_dep["JDEP"] = pd.to_datetime(df_sillons_dep["JDEP"], format="%d/%m/%Y", errors="coerce")

# Fonction pour convertir une heure en minutes
def convert_hour_to_minutes(hour_str):
    """Convertit une heure HH:MM en minutes depuis minuit."""
    if pd.isna(hour_str) or not isinstance(hour_str, str):
        return None  # Valeur invalide
    try:
        h, m = map(int, hour_str.split(":"))
        return (h * 60) + m  # Convertir en minutes
    except ValueError:
        return None  # Si le format est incorrect

# Dictionnaires pour stocker les temps d'arrivée et de départ en minutes
t_a = {}
t_d = {}

# Traitement des arrivées
for _, row in df_sillons_arr.iterrows():
    train_id = row["n°TRAIN"]
    date_arr = row["JARR"]
    heure_arr = convert_hour_to_minutes(row["HARR"])  # Conversion heure → minutes

    if pd.notna(date_arr) and heure_arr is not None:
        days_since_ref = (date_arr - base_time).days  # Nombre de jours depuis le 08/08
        minutes_since_ref = (days_since_ref * 1440) + heure_arr  # Ajout des minutes

        # Création d'un ID unique : Train_ID_Date, car certains trains portant le même ID passent sur des jours différents
        train_id_unique = f"{train_id}_{date_arr.strftime('%d')}"
        
        t_a[train_id_unique] = minutes_since_ref

        print(f"Train {train_id_unique} : JARR = {date_arr.date()}, minutes écoulées = {minutes_since_ref}")

# Traitement des départs
for _, row in df_sillons_dep.iterrows():
    train_id = row["n°TRAIN"]
    date_dep = row["JDEP"]
    heure_dep = convert_hour_to_minutes(row["HDEP"])  # Conversion heure → minutes

    if pd.notna(date_dep) and heure_dep is not None:
        days_since_ref = (date_dep - base_time).days  # Nombre de jours depuis le 08/08
        minutes_since_ref = (days_since_ref * 1440) + heure_dep  # Ajout des minutes

        # Création d'un ID unique : Train_ID_Date
        train_id_unique = f"{train_id}_{date_dep.strftime('%d')}"
        
        t_d[train_id_unique] = minutes_since_ref

        print(f"Train {train_id_unique} : JDEP = {date_dep.date()}, minutes écoulées = {minutes_since_ref}")

# Vérification finale
print("Vérification du contenu final de t_a :")
for key, value in t_a.items():
    print(f"Train {key}: {value} min")

print("Vérification du contenu final de t_d :")
for key, value in t_d.items():
    print(f"Train {key}: {value} min")


Train 412202_08 : JARR = 2022-08-08, minutes écoulées = 1248
Train 431018_08 : JARR = 2022-08-08, minutes écoulées = 801
Train 431246_08 : JARR = 2022-08-08, minutes écoulées = 664
Train 44951_08 : JARR = 2022-08-08, minutes écoulées = 1130
Train 450237_08 : JARR = 2022-08-08, minutes écoulées = 707
Train 489016_08 : JARR = 2022-08-08, minutes écoulées = 1415
Train 54234_08 : JARR = 2022-08-08, minutes écoulées = 1321
Train 55390_08 : JARR = 2022-08-08, minutes écoulées = 1429
Train 56135_08 : JARR = 2022-08-08, minutes écoulées = 1288
Train 412023_09 : JARR = 2022-08-09, minutes écoulées = 1567
Train 41264_09 : JARR = 2022-08-09, minutes écoulées = 2793
Train 431802_09 : JARR = 2022-08-09, minutes écoulées = 1620
Train 44222_09 : JARR = 2022-08-09, minutes écoulées = 2533
Train 44250_09 : JARR = 2022-08-09, minutes écoulées = 2058
Train 44865_09 : JARR = 2022-08-09, minutes écoulées = 2416
Train 450237_09 : JARR = 2022-08-09, minutes écoulées = 2147
Tra

In [47]:
model = Model("SNCF JALON 1")

M_arr = 3  # Nombre de tâches à effectuer sur chaque train d'arrivée
M_dep = 4  # Nombre de tâches à effectuer sur chaque train de départ
S = 60*24*7     # Nombre de minutes dans une semaine    

liste_id_train_arrivee = t_a.keys()
liste_id_train_depart = t_d.keys()

taches_arrivee = [1,2,3]
taches_depart = [1,2,3,4]

# Temps de début de la tâche m sur le train d'arrivée n, en minute, comptée à partir du lundi 8 Aout 2022 00:00
t_arr = {(m,id_train_arr) : model.addVar(vtype = GRB.INTEGER, name = "t") for m in taches_arrivee for id_train_arr in liste_id_train_arrivee} 

# Temps de début de la tâche m sur le train de départ n, en minute, comptée à partir du lundi 8 Aout 2022 00:00
t_dep = {(m,id_train_dep) : model.addVar(vtype = GRB.INTEGER, name = "t") for m in taches_depart for id_train_dep in liste_id_train_depart} 

In [48]:
T_arr = {1:15, 2:45, 3:15}       # Durée des tâches sur les trains d'arrivée
T_dep = {1:15, 2:150, 3:15, 4:20}       # Durée des tâches sur les trains de départ


In [49]:
df_correspondance = pd.read_excel(file, sheet_name="Correspondances")

D = init_dict_correspondance(df_correspondance)

**Contraintes de temporalité des tâches sur un même train et respect des heures de départ et d'arrivée**

In [50]:
for id_train_arr in liste_id_train_arrivee:
    model.addConstr(t_arr[(1,id_train_arr)] >= t_a[id_train_arr])                
    for m in taches_arrivee[:-1]:
        model.addConstr(t_arr[(m,id_train_arr)] + T_arr[m] <= t_arr[(m+1,id_train_arr)]) 
    
        
for id_train_dep in liste_id_train_depart:
    M_dep = taches_depart[-1]               
    model.addConstr(t_dep[(M_dep,id_train_dep)] + T_dep[M_dep] <= t_d[id_train_dep])    
    for m in taches_depart[:-1]:
        model.addConstr(t_dep[(m,id_train_dep)] + T_dep[m] <= t_dep[(m+1,id_train_dep)]) 


**Contrainte permettant d'avoir au plus un wagon par machine à chaque instant**

In [ ]:
M_big = 100000  # Une grande constante, à ajuster en fonction de tes données
delta_arr = {}

for id_arr_1 in liste_id_train_arrivee:
    for id_arr_2 in liste_id_train_arrivee:
        if id_arr_1 != id_arr_2:
            delta_arr[(id_arr_1,id_arr_2)] = model.addVar(vtype=GRB.BINARY, name=f"delta_arr_{id_arr_1}_{id_arr_2}")
            
            # Si delta = 1, alors id_arr_2 se termine avant id_arr_1
            model.addConstr(t_arr[(3, id_arr_2)] + T_arr[3] <= t_arr[(3, id_arr_1)] + (1 - delta_arr[(id_arr_1,id_arr_2)]) * M_big)
            
            # Si delta = 0, alors id_arr_1 se termine avant id_arr_2
            model.addConstr(t_arr[(3, id_arr_2)] >= t_arr[(3, id_arr_1)] + T_arr[3] - delta_arr[(id_arr_1,id_arr_2)] * M_big)

delta_dep = {}

for m_dep in [1, 3]:
    for id_dep_1 in liste_id_train_depart:
        for id_dep_2 in liste_id_train_depart:
            if id_dep_1 != id_dep_2:
                delta_dep[(m_dep,id_dep_1,id_dep_2)] = model.addVar(vtype=GRB.BINARY, name=f"delta_dep_{m_dep}_{id_dep_1}_{id_dep_2}")
                
                # Si delta = 1, alors id_dep_2 se termine avant id_dep_1
                model.addConstr(t_dep[(m_dep, id_dep_2)] + T_dep[m_dep] <= t_dep[(m_dep, id_dep_1)] + (1 - delta_dep[(m_dep,id_dep_1,id_dep_2)]) * M_big)
                
                # Si delta = 0, alors id_dep_1 se termine avant id_dep_2
                model.addConstr(t_dep[(m_dep, id_dep_2)] >= t_dep[(m_dep, id_dep_1)] + T_dep[m_dep] - delta_dep[(m_dep,id_dep_1,id_dep_2)] * M_big)


KeyError: '431018_08'

**Contrainte de respect des horaires d'ouvertures des voies et d'utilisation des machines**

In [ ]:
Limites = np.array([5*60,13*60,(5*24+13)*60,(5*24+21)*60,(6*24+13)*60,(6*24+21)*60,(7*24+5)*60,(7*24+13)*60])      # Horaires limites de la disponibilité du chantier de formation et des machines (en minutes)


M_big = 100000  # Une grande constante, à ajuster en fonction de tes données

delta_lim_arr = {}

for id_arr in liste_id_train_arrivee:
    delta_lim_arr[id_arr] = model.addVars(5, vtype=GRB.BINARY, name=f"delta_arr_{id_arr}")  # 5 cas possibles

    # Cas 1 : Avant la première limite
    model.addConstr(t_arr[(3, id_arr)] <= Limites[0] - T_arr[3] + (1 - delta_lim_arr[id_arr][0]) * M_big)

    # Cas 2 : Entre Limites[1] et Limites[2]
    model.addConstr(t_arr[(3, id_arr)] >= Limites[1] - delta_lim_arr[id_arr][1] * M_big)  # Limite inf
    model.addConstr(t_arr[(3, id_arr)] <= Limites[2] - T_arr[3] + (1 - delta_lim_arr[id_arr][1]) * M_big)  # Limite sup

    # Cas 3 : Entre Limites[3] et Limites[4]
    model.addConstr(t_arr[(3, id_arr)] >= Limites[3] - delta_lim_arr[id_arr][2] * M_big)  # Limite inf
    model.addConstr(t_arr[(3, id_arr)] <= Limites[4] - T_arr[3] + (1 - delta_lim_arr[id_arr][2]) * M_big)  # Limite sup

    # Cas 4 : Entre Limites[5] et Limites[6]
    model.addConstr(t_arr[(3, id_arr)] >= Limites[5] - delta_lim_arr[id_arr][3] * M_big)  # Limite inf
    model.addConstr(t_arr[(3, id_arr)] <= Limites[6] - T_arr[3] + (1 - delta_lim_arr[id_arr][3]) * M_big)  # Limite sup

    # Cas 5 : Après la dernière limite
    model.addConstr(t_arr[(3, id_arr)] >= Limites[7] - delta_lim_arr[id_arr][4] * M_big)

    # Une seule de ces conditions peut être vraie
    model.addConstr(delta_lim_arr[id_arr][0] + delta_lim_arr[id_arr][1] + delta_lim_arr[id_arr][2] + delta_lim_arr[id_arr][3] + delta_lim_arr[id_arr][4] == 1)

delta_lim_dep = {}

for m_dep in [1, 2, 3]:
    for id_dep in liste_id_train_depart:
        delta_lim_dep[(m_dep,id_dep)] = model.addVars(5, vtype=GRB.BINARY, name=f"delta_dep_{id_dep}")  # 5 cas possibles

        # Cas 1 : Avant la première limite
        model.addConstr(t_dep[(m_dep, id_dep)] <= Limites[0] - T_dep[m_dep] + (1 - delta_lim_dep[(m_dep,id_dep)][0]) * M_big)

        # Cas 2 : Entre Limites[1] et Limites[2]
        model.addConstr(t_dep[(m_dep, id_dep)] >= Limites[1] - delta_lim_dep[(m_dep,id_dep)][1] * M_big)  # Limite inf
        model.addConstr(t_dep[(m_dep, id_dep)] <= Limites[2] - T_dep[m_dep] + (1 - delta_lim_dep[(m_dep,id_dep)][1]) * M_big)  # Limite sup

        # Cas 3 : Entre Limites[3] et Limites[4]
        model.addConstr(t_dep[(m_dep, id_dep)] >= Limites[3] - delta_lim_dep[(m_dep,id_dep)][2] * M_big)  # Limite inf
        model.addConstr(t_dep[(m_dep, id_dep)] <= Limites[4] - T_dep[m_dep] + (1 - delta_lim_dep[(m_dep,id_dep)][2]) * M_big)  # Limite sup

        # Cas 4 : Entre Limites[5] et Limites[6]
        model.addConstr(t_dep[(m_dep, id_dep)] >= Limites[5] - delta_lim_dep[(m_dep,id_dep)][3] * M_big)  # Limite inf
        model.addConstr(t_dep[(m_dep, id_dep)] <= Limites[6] - T_dep[m_dep] + (1 - delta_lim_dep[(m_dep,id_dep)][3]) * M_big)  # Limite sup

        # Cas 5 : Après la dernière limite
        model.addConstr(t_dep[(m_dep, id_dep)] >= Limites[7] - delta_lim_dep[(m_dep,id_dep)][4] * M_big)

        # Une seule de ces conditions peut être vraie
        model.addConstr(delta_lim_dep[(m_dep,id_dep)][0] + delta_lim_dep[(m_dep,id_dep)][1] + delta_lim_dep[(m_dep,id_dep)][2] + delta_lim_dep[(m_dep,id_dep)][3] + delta_lim_dep[(m_dep,id_dep)][4] == 1) 


**Contrainte de succession des tâches sur les trains d'arrivées et des tâches sur les trains de départ en tenant compte de la correspondance des wagons**

In [ ]:
for id_dep in liste_id_train_depart:
    for id_arr in D[id_dep]:
        model.addConstr(t_dep[(1,id_dep)] >= t_arr[(3,id_arr)] + T_arr[3])

In [ ]:
# -- Choix d'un paramétrage d'affichage minimaliste --
model.params.outputflag = 0 # mode muet
# -- Mise à jour du modèle  --
model.update()
# -- Résolution --
model.optimize()

if model.status == GRB.INFEASIBLE:
    print("Le modèle n'a pas de solution")
elif model.status == GRB.UNBOUNDED:
    print("Le modèle est non borné")
else:
    print("Ca marche Bébou")
if model.status == GRB.OPTIMAL:
    for (m, n), var in t_arr.items():
        print(f"Tâche {m} - Train {n} : {var.x}")
    for (m, n), var in t_dep.items():
        print(f"Tâche {m} - Train {n} : {var.x}")


Ca marche Bébou
Tâche 1 - Train 412202_08 : -0.0
Tâche 1 - Train 431018_08 : -0.0
Tâche 1 - Train 431246_08 : -0.0
Tâche 1 - Train 44951_08 : -0.0
Tâche 1 - Train 450237_08 : -0.0
Tâche 1 - Train 489016_08 : -0.0
Tâche 1 - Train 54234_08 : -0.0
Tâche 1 - Train 55390_08 : -0.0
Tâche 1 - Train 56135_08 : -0.0
Tâche 1 - Train 412023_09 : -0.0
Tâche 1 - Train 41264_09 : -0.0
Tâche 1 - Train 431802_09 : -0.0
Tâche 1 - Train 44222_09 : -0.0
Tâche 1 - Train 44250_09 : -0.0
Tâche 1 - Train 44865_09 : -0.0
Tâche 1 - Train 450237_09 : -0.0
Tâche 1 - Train 471003_09 : -0.0
Tâche 1 - Train 47262_09 : -0.0
Tâche 1 - Train 489026_09 : -0.0
Tâche 1 - Train 54234_09 : -0.0
Tâche 1 - Train 55390_09 : -0.0
Tâche 1 - Train 55398_09 : -0.0
Tâche 1 - Train 56135_09 : -0.0
Tâche 1 - Train 56730_09 : -0.0
Tâche 1 - Train 58706_09 : -0.0
Tâche 1 - Train 412023_10 : -0.0
Tâche 1 - Train 412202_10 : -0.0
Tâche 1 - Train 41264_10 : -0.0
Tâche 1 - Train 431020_10 : -0.0
Tâche 1 - Train 431110_10 : -0.0
Tâche 1 - 